In [4]:
import sys
sys.path.append("../")
from tuneavideo.pipelines.pipeline_tuneavideo import TuneAVideoPipeline
from tuneavideo.pipelines.pipeline_tuneavideo_video2video import TuneAVideoVideo2VideoPipeline
from tuneavideo.pipelines import pipeline_tuneavideo_video2video

from tuneavideo.models.unet import UNet3DConditionModel
from tuneavideo.util import save_videos_grid, read_gif, save_as_gif
import torch
from einops import rearrange
import einops
import decord
from matplotlib import pyplot as plt
from PIL import Image
from moviepy import editor
import numpy as np
decord.bridge.set_bridge('torch')

In [5]:
def to_np_video(x):
    x = x.squeeze(0)
    print(x.shape)
    x = rearrange(x, "c t h w -> t h w c")
    print(x.shape)
    x = (x * 255).numpy().astype(np.uint8)
    return x


In [6]:
# 無学習
unet = UNet3DConditionModel.from_pretrained_2d("../checkpoints/sd15/unet").half().cuda()

In [7]:
pipe = TuneAVideoPipeline.from_pretrained("../checkpoints/sd15", unet=unet, torch_dtype=torch.float16).to("cuda")
pipe.unet.enable_xformers_memory_efficient_attention()

In [8]:
# posi = "anime 1girl tilting her head, white hair blue eyes, masterpiece, best quality, 8k, detailed anime style"
# nega = "3d, flat shading, flat color, retro style, 1980s, 1990s, 2000s, 2005 2006 2007 2008 2009 2010 2011 2012 2013 2014 2015 2016 2017 2018 2019, lowres, bad anatomy, bad hands, text, error, missing fingers, extra digit, fewer digits, cropped, worst quality, low quality, normal quality, jpeg artifacts, signature, watermark, username, blurry, artist name, inaccurate limb"
posi = "a firefighter is dancing"
nega = ""

strength = 0.7
seed = 1
fps = 8
video_length = 8

In [11]:
# t2v
torch.manual_seed(seed)
v = pipe(prompt=posi,negative_prompt=nega,video_length=8, height=512, width=512, num_inference_steps=50, guidance_scale=7.5, temporal_guidance_scale=4).videos
v_np = to_np_video(v)
clip = editor.ImageSequenceClip(list(v_np), fps=8)
clip.ipython_display()

100%|██████████| 50/50 [01:48<00:00,  2.18s/it]


torch.Size([3, 8, 512, 512])
torch.Size([8, 512, 512, 3])
Moviepy - Building video __temp__.mp4.
Moviepy - Writing video __temp__.mp4



Moviepy - Done !
Moviepy - video ready __temp__.mp4


In [ ]:
unet_bin = torch.load("/mnt/disks/disk_main/projects/Tune-A-Video/outputs/yurucamp_stamp/1qkv2qkv_16000_24e5/checkpoint-2000/pytorch_model.bin")

In [ ]:
model_id = "../outputs/tilting_head/2023-02-04T12-46-15"
unet = UNet3DConditionModel.from_pretrained(model_id, subfolder='unet', torch_dtype=torch.float16).to('cuda')

In [ ]:
unet.load_state_dict(unet_bin)

In [ ]:
video = decord.VideoReader("../data/tilting_head/tilting_head.mp4")

In [ ]:
# v2v
torch.manual_seed(0)
v = pipe(prompt=posi,negative_prompt=nega, video=video[:8],strength=strength,height=512, width=512, num_inference_steps=50, guidance_scale=7.5, fix_seed_across_frames=False).videos
# save_videos_grid(v, f"../data/out/tmp.gif")
v_np = to_np_video(v)
clip = editor.ImageSequenceClip(list(v_np), fps=8)
clip.ipython_display()

In [ ]:
clip.write_videofile("../data/out/shake_head_overfit.mp4")

In [ ]:
clip.write_videofile("../data/tilting_head/out/v2v_strength_07_white_hair.mp4")

In [ ]:
t = torch.randn(1, 3, 30, 30)
einops.repeat(t, "b c h w -> b c f h w", f=10).shape

In [ ]:
images = [Image.fromarray(image.numpy()) for image in video]
save_as_gif(images, "../data/yor_short.gif", duration=330)

In [ ]:
v = video[0]
v = rearrange(v, "c f h w -> f h w c")
plt.imshow(v[3].cpu().numpy())

In [ ]:
torch.cuda.empty_cache()

In [ ]:
pipe.scheduler.num_inference_steps

In [ ]:
num_inference_steps = 50
strength = 0.5

pipe.scheduler.set_timesteps(num_inference_steps, device=pipe.device)

def prepare_init_latent(pipe, video_path, index_list:list, strength):
    vr = decord.VideoReader("../data/yor.mp4", width=512, height=512)
    video = vr.get_batch(index_list)
    video = rearrange(video, "b h w c -> b c h w")
    video = video.to(pipe.device).half()
    latent = pipe.vae.encode(video).latent_dist.sample()
    latent = 0.18215 * latent
    noise = torch.randn(latent.shape, device=pipe.device, dtype=torch.float16)

    init_timestep = int(pipe.scheduler.num_inference_steps * strength)
    t_start = pipe.scheduler.num_inference_steps - init_timestep
    timesteps = pipe.scheduler.timesteps[t_start:]

    latent = pipe.scheduler.add_noise(latent, noise, timesteps[:1])
    print(latent.shape)
    
prepare_init_latent(pipe, "../data/yor.mp4", [0, 1, 2, 3,4], 0.5)


In [ ]:
# TODO: This code may degrade image quality
def read_gif(path) -> list[Image.Image]:
    gif_image = Image.open(path)
    images = []
    for index in range(gif_image.n_frames):
        gif_image.seek(index)
        images.append(gif_image.copy().convert("RGB"))

    return images

# TODO: This code may degrade image quality
def save_as_gif(images: list[Image.Image], path, duration=500, loop=0):
    images[0].save(path, save_all=True, append_images=images[1:], duration=duration, loop=loop)
